In [27]:
import json
import pandas as pd
from pathlib import Path

# Train a german ULMFIT language model from scratch

Inspired by: https://github.com/insightfactory-app/ULMFIT-Persian/blob/master/Fast_ai_build_persian_pretrained_model.ipynb

## Prepare Wikipedia Dump

Find a mirror for the Wikipedia XML-dump in the prefered language here https://wikimedia.mirror.us.dev/backup-index.html (scroll down for the different languages) and download the dump with `wget`.

In [2]:
!wget -P data/ https://ftp.acc.umu.se/mirror/wikimedia.org/dumps/dewiki/20200920/dewiki-20200920-pages-articles-multistream.xml.bz2

--2020-09-28 19:39:30--  https://ftp.acc.umu.se/mirror/wikimedia.org/dumps/dewiki/20200920/dewiki-20200920-pages-articles-multistream.xml.bz2
Auflösen des Hostnamens ftp.acc.umu.se (ftp.acc.umu.se)… 194.71.11.173, 194.71.11.165
Verbindungsaufbau zu ftp.acc.umu.se (ftp.acc.umu.se)|194.71.11.173|:443 … verbunden.
HTTP-Anforderung gesendet, auf Antwort wird gewartet … 200 OK
Länge: 5950195273 (5,5G) [application/x-bzip2]
Wird in »dewiki-20200920-pages-articles-multistream.xml.bz2.1« gespeichert.

dewiki-20200920-pag 100%[===================>]   5,54G  4,90MB/s    in 17m 35s 

2020-09-28 19:57:05 (5,38 MB/s) - »dewiki-20200920-pages-articles-multistream.xml.bz2.1« gespeichert [5950195273/5950195273]



Unpack the dump using `bzip2`:

In [3]:
!bzip2 -d "data/dewiki-20200920-pages-articles-multistream.xml.bz2"

Clone the wikiextractor repo (https://github.com/attardi/wikiextractor). Use Python 3.6 environment to avoid Depreciation Warnings!

In [10]:
!git clone https://github.com/attardi/wikiextractor.git

Cloning into 'wikiextractor'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 661 (delta 0), reused 0 (delta 0), pack-reused 657
Receiving objects: 100% (661/661), 1.27 MiB | 2.73 MiB/s, done.
Resolving deltas: 100% (374/374), done.


In [ ]:
!pip install wikiextractor

In [15]:
# -s, --sections        preserve sections
# --json                write output in json format instead of the default one
# --no-templates        Do not expand templates
# -q, --quiet           suppress reporting progress info
# -o OUTPUT, --output OUTPUT
#                        directory for extracted files (or '-' for dumping to
#                        stdout)

In [16]:
!python -m wikiextractor.WikiExtractor --json --no_templates -q -o data/dewiki data/dewiki-20200920-pages-articles-multistream.xml

/Users/florian/miniconda3/envs/fastaidev/bin/python: No module named wikiextractor.WikiExtractor


In [40]:
from fastai.basics import *
import re


def get_wiki(path,lang):
    name = f'{lang}wiki'
    if (path/name).exists():
        print(f"{path/name} already exists; not downloading")
        return

    xml_fn = f"{lang}wiki-latest-pages-articles.xml"
    zip_fn = f"{xml_fn}.bz2"

    if not (path/xml_fn).exists():
        print("downloading...")
        download_url(f'https://dumps.wikimedia.org/{name}/latest/{zip_fn}', path/zip_fn)
        print("unzipping...")
        bunzip(path/zip_fn)

    with working_directory(path):
        if not (path/'wikiextractor').exists(): os.system('git clone https://github.com/attardi/wikiextractor.git')
        print("extracting...")
        os.system("python wikiextractor.WikiExtractor.py --processes 4 --no_templates " +
            f"--min_text_length 1800 --filter_disambig_pages --log_file log -b 100G -q {xml_fn}")
    shutil.move(str(path/'text/AA/wiki_00'), str(path/name))
    shutil.rmtree(path/'text')


def split_wiki(path,lang):
    dest = path/'docs'
    name = f'{lang}wiki'
    if dest.exists():
        print(f"{dest} already exists; not splitting")
        return dest

    dest.mkdir(exist_ok=True, parents=True)
    title_re = re.compile(rf'<doc id="\d+" url="https://{lang}.wikipedia.org/wiki\?curid=\d+" title="([^"]+)">')
    lines = (path/name).open()
    f=None

    for i,l in enumerate(lines):
        if i%100000 == 0: print(i)
        if l.startswith('<doc id="'):
            title = title_re.findall(l)[0].replace('/','_')
            if len(title)>150: continue
            if f: f.close()
            f = (dest/f'{title}.txt').open('w')
        else: f.write(l)
    f.close()
    return dest

Inspired by https://github.com/fastai/course-nlp/blob/master/nlputils.py

In [48]:
from fastai.basics import *
import re

In [45]:
dump = 'https://ftp.acc.umu.se/mirror/wikimedia.org/dumps/dewiki/20200920/dewiki-20200920-pages-articles.xml.bz2'

In [57]:
path = Path('data')
zip_fn = dump.split('/')[-1]
zip_fn

'dewiki-20200920-pages-articles.xml.bz2'

In [58]:
xml_fn = zip_fn[:-4]
xml_fn

'dewiki-20200920-pages-articles.xml'

In [60]:
name = xml_fn.split('-')[0]
name

'dewiki'

In [47]:
download_url(dump, path/zip_fn)

In [63]:
bunzip(path/zip_fn)

In [71]:
# run in python 3.6 env to avoid regex warnings
#python -m wikiextractor.WikiExtractor --processes 4 --no_templates \
#--min_text_length 1800 --filter_disambig_pages --log_file log -b 100G -q dewiki-20200920-pages-articles.xml

python: can't open file 'wikiextractor.extract.py': [Errno 2] No such file or directory


In [72]:
shutil.move(str(path/'text/AA/wiki_00'), str(path/name))
shutil.rmtree(path/'text')

In [82]:
def split_wiki(path,lang):
    dest = path/'docs'
    name = f'{lang}wiki'
    if dest.exists():
        print(f"{dest} already exists; not splitting")
        return dest

    dest.mkdir(exist_ok=True, parents=True)
    title_re = re.compile(rf'<doc id="\d+" url="https://{lang}.wikipedia.org/wiki\?curid=\d+" title="([^"]+)">')
    lines = (path/name).open()
    f=None

    for i,l in enumerate(lines):
        if i%100000 == 0: print(i)
        if l.startswith('<doc id="'):
            title = title_re.findall(l)[0].replace('/','_')
            if len(title)>150: continue
            if f: f.close()
            f = (dest/f'{title}.txt').open('w')
        elif l.startswith('</doc>'): pass
        else: f.write(l)
    f.close()
    return dest

In [83]:
split_wiki(path,lang)

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000
5800000
5900000
6000000
6100000
6200000
6300000
6400000
6500000
6600000
6700000
6800000
6900000
7000000
7100000
7200000
7300000
7400000
7500000
7600000
7700000
7800000
7900000
8000000
8100000
8200000
8300000
8400000
8500000
8600000
8700000
8800000
8900000
9000000
9100000
9200000
9300000
9400000
9500000
9600000
9700000
9800000
9900000
10000000
10100000
10200000
10300000
10400000
10500000
10600000
10700000
10800000
10900000
11000000
11100000
11200000
11300000
11400000
11500000
11600000
11700000
11800000
11900000
12000000
12100000
12200000
12300000

Path('data/docs')